###La base de datos de airbnb berlin es un dataset que muestra las viviendas dedicadas a acoger personas de todas partes 
de europa y del mundo en la ciudad de berlin. Estos datos fueron scrapeados en el 2008 z estan constituidos de 96 columnas
con 22552 registros.

In [274]:
#Importamos todas las librerias necesarias para trabajar con la limpieza
import pandas as pd
import numpy as np
from scipy import stats 
import re
import string

In [275]:
#Observamos un poco la estructura de nuestra base de datos
Airbnb_berlin = pd.read_csv('listings_summary.csv')
Airbnb_berlin.head(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2015,https://www.airbnb.com/rooms/2015,20181107122246,2018-11-07,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,none,It is located in the former East Berlin area o...,...,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76
1,2695,https://www.airbnb.com/rooms/2695,20181107122246,2018-11-07,Prenzlauer Berg close to Mauerpark,NaN,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,none,NaN,...,t,NaN,NaN,f,f,flexible,f,f,1,1.42
2,3176,https://www.airbnb.com/rooms/3176,20181107122246,2018-11-07,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor (68m2) apartment on Kollwitzplatz/ P...,This beautiful first floor apartment is situa...,none,The neighbourhood is famous for its variety of...,...,t,NaN,NaN,t,f,strict_14_with_grace_period,f,f,1,1.25
3,3309,https://www.airbnb.com/rooms/3309,20181107122246,2018-11-07,BerlinSpot Schöneberg near KaDeWe,First of all: I prefer short-notice bookings. ...,"Your room is really big and has 26 sqm, is ver...",First of all: I prefer short-notice bookings. ...,none,"My flat is in the middle of West-Berlin, direc...",...,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,0.39
4,7071,https://www.airbnb.com/rooms/7071,20181107122246,2018-11-07,BrightRoom with sunny greenview!,Cozy and large room in the beautiful district ...,"The BrightRoom is an approx. 20 sqm (215ft²), ...",Cozy and large room in the beautiful district ...,none,"Great neighborhood with plenty of Cafés, Baker...",...,t,NaN,NaN,f,f,moderate,f,f,1,1.75


In [276]:
#Verificamos las filas y columnas
Airbnb_berlin.shape

(22552, 96)

In [ ]:
"""La base de datos contiene 22,552 registros y 96 columnas, las cuales describen los departamentos y hogares que se ponen
a disposicion de alquiler temporal a traves de la plataforma airbnb. Se aprecia las diferentes zonas y localidades donde
se encuentran los mismos, asi como tambien el precio por dia, semanal y mensual. Se observa ademas el tipo de propiedad
si es apartamento, condominio, hostal, aparta hotel, etc. La estructura de los apartamentos y ademas los reviews."""

In [277]:
#Informaciones sobre las columnas, el tipo de datos y valores nulos
Airbnb_berlin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 96 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                22552 non-null  int64  
 1   listing_url                       22552 non-null  object 
 2   scrape_id                         22552 non-null  int64  
 3   last_scraped                      22552 non-null  object 
 4   name                              22493 non-null  object 
 5   summary                           21589 non-null  object 
 6   space                             14020 non-null  object 
 7   description                       22349 non-null  object 
 8   experiences_offered               22552 non-null  object 
 9   neighborhood_overview             11540 non-null  object 
 10  notes                             7215 non-null   object 
 11  transit                           13036 non-null  object 
 12  acce

In [278]:
#Verificacion de valores duplicados en el dataset
Airbnb_berlin.duplicated().value_counts()

#Se observa que no existen valores duplicados

False    22552
dtype: int64

In [279]:
#Observacion de valores nulos totales para cada columna en el dataset
NA = Airbnb_berlin.isna().sum()
NA[NA > 0]

name                              59
summary                          963
space                           8532
description                      203
neighborhood_overview          11012
notes                          15337
transit                         9516
access                         11715
interaction                    12146
house_rules                    11103
thumbnail_url                  22552
medium_url                     22552
xl_picture_url                 22552
host_name                         26
host_since                        26
host_location                    116
host_about                     11363
host_response_time             12894
host_response_rate             12895
host_acceptance_rate           22552
host_is_superhost                 26
host_thumbnail_url                26
host_picture_url                  26
host_neighbourhood              5094
host_listings_count               26
host_total_listings_count         26
host_has_profile_pic              26
h

In [ ]:
"""Hay columnas que se podrian eliminar debido a que no serian util a tomar en cuenta para futuros analisis, y ademas
hay una gran cantidad de columnas que tienen mas del 50% de los registros nulos, las cuales resultan pocos significantes
para analisis posterioes"""

In [280]:
#Se puede echar un vistazo a las columnas del dataframe
Lista_Columnas = Airbnb_berlin.columns.to_list()
print(Lista_Columnas)

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price', '

In [281]:
def eliminar_columna(df):
    """
        Eliminamos las columnas con valores nulos muy excesivos con mas del 
        50% de los registros y ademas las columnas innecesarias

    """

    df_airbnb = Airbnb_berlin.copy() #siempre hacer una copia del df cuando este trabajando con una funcion

    df_airbnb.drop(['neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 
                    'medium_url', 'xl_picture_url', 'host_about', 'host_response_time', 'host_response_rate', 
                    'host_acceptance_rate', 'square_feet', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 
                    'license', 'jurisdiction_names', 'listing_url', 'scrape_id', 'last_scraped', 'host_id', 'host_url', 
                    'host_name', 'host_since', 'smart_location', 'market', 'country_code', 'country', 'latitude', 'longitude',
                    'calendar_updated', 'calendar_last_scraped', 'first_review', 'last_review', 'host_listings_count',
                    'amenities', 'state', 'city', 'neighbourhood_cleansed', 'street', 'host_verifications', 
                    'experiences_offered', 'picture_url', 'host_thumbnail_url', 'host_picture_url',
                    'neighbourhood_group_cleansed', 'zipcode'], axis=1, inplace=True)
            
    
    return df_airbnb
data_54 = eliminar_columna(Airbnb_berlin)

In [282]:
#drop_columna(Airbnb_berlin)
data_54 = eliminar_columna(Airbnb_berlin)
data_54.shape

(22552, 46)

In [283]:
NA_data_copy = data_54.isna().sum()
NA_data_copy[NA_data_copy > 0]

name                             59
summary                         963
space                          8532
description                     203
host_location                   116
host_is_superhost                26
host_neighbourhood             5094
host_total_listings_count        26
host_has_profile_pic             26
host_identity_verified           26
neighbourhood                  1131
bathrooms                        32
bedrooms                         18
beds                             40
review_scores_rating           4389
review_scores_accuracy         4414
review_scores_cleanliness      4411
review_scores_checkin          4432
review_scores_communication    4418
review_scores_location         4431
review_scores_value            4435
reviews_per_month              3914
dtype: int64

In [284]:
print(Airbnb_berlin.columns.to_list())

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price', '

In [285]:
#Vamos a mostrar los porcentajes de valores nulos de las columnas restantes
def porcentajes_na(df):
   
    nan_value_percentage = (df.isna().sum()/data_54.shape[0]).sort_values(ascending = False)

    # retornamos solos las columnas con valores nulos
    return nan_value_percentage[nan_value_percentage > 0.0]

porcentajes_na(data_54)

space                          0.378326
host_neighbourhood             0.225878
review_scores_value            0.196657
review_scores_checkin          0.196524
review_scores_location         0.196479
review_scores_communication    0.195903
review_scores_accuracy         0.195725
review_scores_cleanliness      0.195592
review_scores_rating           0.194617
reviews_per_month              0.173554
neighbourhood                  0.050151
summary                        0.042701
description                    0.009001
host_location                  0.005144
name                           0.002616
beds                           0.001774
bathrooms                      0.001419
host_identity_verified         0.001153
host_has_profile_pic           0.001153
host_is_superhost              0.001153
host_total_listings_count      0.001153
bedrooms                       0.000798
dtype: float64

In [292]:
#Reemplazamos los valores nulos de bed, bedrooms y bathrooms con la misma media ya que son similares y son de tipo flotantes
def na_fill_rooms(df):
    
    data_55 = data_54.copy()
    na_float_columns = ['beds', 'bedrooms', 'bathrooms']
    
    data_55[na_float_columns] = data_55[na_float_columns].replace('NaN', np.nan).fillna(1.29)
    
    return data_55

data_55 = na_fill_rooms(data_54)

In [293]:
porcentajes_na(data_55)

space                          0.378326
host_neighbourhood             0.225878
review_scores_value            0.196657
review_scores_checkin          0.196524
review_scores_location         0.196479
review_scores_communication    0.195903
review_scores_accuracy         0.195725
review_scores_cleanliness      0.195592
review_scores_rating           0.194617
reviews_per_month              0.173554
neighbourhood                  0.050151
summary                        0.042701
description                    0.009001
host_location                  0.005144
name                           0.002616
host_identity_verified         0.001153
host_total_listings_count      0.001153
host_is_superhost              0.001153
host_has_profile_pic           0.001153
dtype: float64

In [297]:
#Reemplazamos los valores nulos de las columnas de los reviews con valores de sus medias respectivas

def na_fill_reviews(df):
    
    data_56 = data_55.copy()

    na_reviews_columns = ['review_scores_value', 'review_scores_checkin', 'review_scores_location', 
                      'review_scores_communication', 'review_scores_accuracy', 'review_scores_cleanliness'] 
    data_56[na_reviews_columns] = data_56[na_reviews_columns].replace('NaN', np.nan).fillna(9.558285)

    
    na_review_rating = ['review_scores_rating']
    data_56[na_review_rating] = data_56[na_review_rating].replace('NaN', np.nan).fillna(94.413863)

    
    na_review_month = ['reviews_per_month']
    data_56[na_review_month] = data_56[na_review_month].replace('NaN', np.nan).fillna(1.135525)

    
    na_total_listing = ['host_total_listings_count']
    data_56[na_total_listing] = data_56[na_total_listing].replace('NaN', np.nan).fillna(2.330685)
    
    return data_56

data_56 = na_fill_reviews(data_55)

In [298]:
porcentajes_na(data_56)

space                     0.378326
host_neighbourhood        0.225878
neighbourhood             0.050151
summary                   0.042701
description               0.009001
host_location             0.005144
name                      0.002616
host_has_profile_pic      0.001153
host_is_superhost         0.001153
host_identity_verified    0.001153
dtype: float64

In [299]:
#Reemplazamos los valores nulos de las columnas de tipo strings por no especificado
def fill_string(df):
    
    data_57 = data_56.copy() 
    
    na_strings_none = ['space', 'host_neighbourhood', 'neighbourhood', 'summary', 'description', 'host_location', 'name']

    for columna in na_strings_none:
        data_57[columna].fillna('Not specified', inplace=True)
        
    return data_57

data_57 = fill_string(data_56)

In [300]:
print(porcentajes_na(data_57))

host_is_superhost         0.001153
host_has_profile_pic      0.001153
host_identity_verified    0.001153
dtype: float64


In [314]:
"""Reemplazamos los valores nulos de las columnas booleanas por falso o el valor cero, antes de reemplazar
los valores debemos convertir los valores booleanos actuales T y F por 1 y 0 respectivamente. Ademas vamos agregar
las demas columnas que no tienen valores nulos pero que si requieren una convercion de valor falso o verdadero a 0 y 1"""

def convert_true_false(df):
    
    data_58 = data_57.copy()
#creamos una lista de las columnas que contienen valores booleanos  
    boolean_columns = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified',
                       'requires_license', 'instant_bookable', 'require_guest_profile_picture', 
                       'require_guest_phone_verification', 'is_location_exact', 'has_availability',
                       'is_business_travel_ready'] 

#reemplazamos los valores nulos de los booleanos por 0
    data_58[boolean_columns] = data_58[boolean_columns].replace('t', '1')
    data_58[boolean_columns] = data_58[boolean_columns].replace('f', '0')
    data_58[boolean_columns] = data_58[boolean_columns].replace('NaN', np.nan).fillna(0)

#convertimos los valores de t y f por 1 y 0 respectivamente y luego lo trabsformamos a entero
    for col in boolean_columns:
     #   data_58[col] = data_58[col].apply(convert_true_false)
        data_58[col] = data_58[col].astype(np.int64)
        
    return data_58
    
#data_54[boolean_columns].isnull().values.any()
data_58 = convert_true_false(data_57)  

In [315]:
#print(porcentajes_na(data_58))
data_58[boolean_columns].isnull().values.any()

False

In [316]:
data_58.head(5)

,id,name,summary,space,description,host_location,host_is_superhost,host_neighbourhood,host_total_listings_count,host_has_profile_pic,...,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,"Key Biscayne, Florida, United States",1,Mitte,4.0,1,...,10.0,9.0,1,0,0,strict_14_with_grace_period,0,0,4,3.76
1,2695,Prenzlauer Berg close to Mauerpark,Not specified,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,"Berlin, Berlin, Germany",0,Prenzlauer Berg,1.0,1,...,10.0,10.0,1,0,0,flexible,0,0,1,1.42
2,3176,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor (68m2) apartment on Kollwitzplatz/ P...,This beautiful first floor apartment is situa...,"Coledale, New South Wales, Australia",0,Prenzlauer Berg,1.0,1,...,10.0,9.0,1,1,0,strict_14_with_grace_period,0,0,1,1.25
3,3309,BerlinSpot Schöneberg near KaDeWe,First of all: I prefer short-notice bookings. ...,"Your room is really big and has 26 sqm, is ver...",First of all: I prefer short-notice bookings. ...,"Berlin, Berlin, Germany",0,Schöneberg,1.0,1,...,9.0,9.0,1,0,0,strict_14_with_grace_period,0,0,1,0.39
4,7071,BrightRoom with sunny greenview!,Cozy and large room in the beautiful district ...,"The BrightRoom is an approx. 20 sqm (215ft²), ...",Cozy and large room in the beautiful district ...,"Berlin, Berlin, Germany",1,Prenzlauer Berg,1.0,1,...,10.0,9.0,1,0,0,moderate,0,0,1,1.75


In [318]:
np.unique(data_58[boolean_columns].values)

array([0, 1], dtype=int64)

In [319]:
#En nuestro data frame tenemos dos variables de precio que son de tipo objeto y debemos convertilas a flotante
price_columns = ['price', 'extra_people']
data_58[price_columns].head()

,price,extra_people
0,$60.00,$28.00
1,$17.00,$0.00
2,$90.00,$20.00
3,$26.00,$18.00
4,$42.00,$24.00


In [328]:
#Debemos eliminar los caracteres y simbolos especiales con regex
def limpiar_precios(df):
    
    data_59 = data_58.copy()

    price_columns = ['price', 'extra_people']

    for col in price_columns:
    # remove "$" and "," chars
        data_59[col] = data_59[col].replace({'\$':''}, regex = True)
        data_59[col] = data_59[col].replace({',':''}, regex = True)

    # convert price to float
        data_59[col] = data_59[col].astype(float)
        
    return data_59

data_59 = limpiar_precios(data_58)

In [329]:
data_59[price_columns].head()

,price,extra_people
0,60.0,28.0
1,17.0,0.0
2,90.0,20.0
3,26.0,18.0
4,42.0,24.0


In [330]:
#Vamos a renombrar las columnas a nuestra comodidad para futuros analisis de modelaje
def cambiando_nombres(df):
    """
    Esta funcion renombras las columnas
    del DataFrame
    """
    data_60 = data_59.copy() #siempre hacer una copia del df cuando este trabajando con un funcion
    data_60.columns = ['ID', 'Nombre_Localidad', 'Opinion_Cliente', 'Espacio', 'Descripcion_Vivienda', 'Ubicacion_Anfitrion',
                       'Anfitrion_acojedor', 'Zona_Vivienda', 'Conteo_Total_Host', 'Anfitrion_Foto_Perfil', 
                       'Identidad_Anfitrion_Verificada', 'Localidad', 'Vivivenda_Ubicacion_Exacta', 
                       'Tipo_Propiedad', 'Tipo_Salas', 'Comodidad', 'Salas_Baños', 'Dormitorios', 'Camas',
                       'Tipos_Camas', 'Precio', 'Invitados_Incluidos', 'Precio_Persona_Extra', 'Noches_Minimas',
                       'Noches_Maximas', 'Esta_Disponible', 'Disponibilidad_1mes', 'Disponibilidad_2meses',
                       'Disponibilidad_3meses', 'Disponibilidad_365', 'Numero_Reseña', 'Puntaje_Nivel',
                       'Calidad_Puntaje', 'Limpieza_Puntaje', 'Chequeo_Entrada_Puntaje', 'Puntaje_Comunicacion',
                       'Ubicacion_Puntaje', 'Valor_Puntaje', 'Requiere_Licencia', 'Reserva_Instantanea',
                       'Disponible_Viaje_Negocios', 'Politica_Cancelacion', 'Cliente_Foto_Perfil', 
                       'Cliente_Telefono_vericicacion', 'Conteo_Lista_Anfitrion_Calculada', 'Reseña_Mensual']

    return data_60

data_60 = cambiando_nombres(data_59)

In [331]:
data_60.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 46 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                22552 non-null  int64  
 1   Nombre_Localidad                  22552 non-null  object 
 2   Opinion_Cliente                   22552 non-null  object 
 3   Espacio                           22552 non-null  object 
 4   Descripcion_Vivienda              22552 non-null  object 
 5   Ubicacion_Anfitrion               22552 non-null  object 
 6   Anfitrion_acojedor                22552 non-null  int64  
 7   Zona_Vivienda                     22552 non-null  object 
 8   Conteo_Total_Host                 22552 non-null  float64
 9   Anfitrion_Foto_Perfil             22552 non-null  int64  
 10  Identidad_Anfitrion_Verificada    22552 non-null  int64  
 11  Localidad                         22552 non-null  object 
 12  Vivi

In [260]:
New_DF_Airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 46 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                22552 non-null  int64  
 1   Nombre_Localidad                  22552 non-null  object 
 2   Opinion_Cliente                   22552 non-null  object 
 3   Espacio                           22552 non-null  object 
 4   Descripcion_Vivienda              22552 non-null  object 
 5   Ubicacion_Anfitrion               22552 non-null  object 
 6   Anfitrion_acojedor                22552 non-null  int64  
 7   Zona_Vivienda                     22552 non-null  object 
 8   Conteo_Total_Host                 22552 non-null  float64
 9   Anfitrion_Foto_Perfil             22552 non-null  int64  
 10  Identidad_Anfitrion_Verificada    22552 non-null  int64  
 11  Localidad                         22552 non-null  object 
 12  Vivi

In [332]:
#Solo Falta limpiar los registros tipo texo del dataset

def limpiar_string(df):
    data_61 = data_60.copy()
    
    columnas_texto = ['Nombre_Localidad', 'Opinion_Cliente', 'Espacio', 'Descripcion_Vivienda', 'Ubicacion_Anfitrion',
                      'Zona_Vivienda', 'Localidad', 'Tipo_Propiedad', 'Tipo_Salas', 'Comodidad', 'Tipos_Camas', 
                      'Disponible_Viaje_Negocios', 'Politica_Cancelacion']

#Debemos eliminar los caracteres y simbolos especiales con regex
    for col in columnas_texto :
        data_61[col] = data_61[col].replace({'\W': " "}, regex = True)
    
    return data_61

data_61 = limpiar_string(data_60)

In [333]:
data_61.head()

,ID,Nombre_Localidad,Opinion_Cliente,Espacio,Descripcion_Vivienda,Ubicacion_Anfitrion,Anfitrion_acojedor,Zona_Vivienda,Conteo_Total_Host,Anfitrion_Foto_Perfil,...,Ubicacion_Puntaje,Valor_Puntaje,Requiere_Licencia,Reserva_Instantanea,Disponible_Viaje_Negocios,Politica_Cancelacion,Cliente_Foto_Perfil,Cliente_Telefono_vericicacion,Conteo_Lista_Anfitrion_Calculada,Reseña_Mensual
0,2015,Berlin Mitte Value Quiet courtyard very central,Great location 30 of 75 sq meters This wood...,A location This Einliegerwohnung is an e...,Great location 30 of 75 sq meters This wood...,Key Biscayne Florida United States,1,Mitte,4.0,1,...,10.0,9.0,1,0,0,strict_14_with_grace_period,0,0,4,3.76
1,2695,Prenzlauer Berg close to Mauerpark,Not specified,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,Berlin Berlin Germany,0,Prenzlauer Berg,1.0,1,...,10.0,10.0,1,0,0,flexible,0,0,1,1.42
2,3176,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor 68m2 apartment on Kollwitzplatz P...,This beautiful first floor apartment is situa...,Coledale New South Wales Australia,0,Prenzlauer Berg,1.0,1,...,10.0,9.0,1,1,0,strict_14_with_grace_period,0,0,1,1.25
3,3309,BerlinSpot Schöneberg near KaDeWe,First of all I prefer short notice bookings ...,Your room is really big and has 26 sqm is ver...,First of all I prefer short notice bookings ...,Berlin Berlin Germany,0,Schöneberg,1.0,1,...,9.0,9.0,1,0,0,strict_14_with_grace_period,0,0,1,0.39
4,7071,BrightRoom with sunny greenview,Cozy and large room in the beautiful district ...,The BrightRoom is an approx 20 sqm 215ft² ...,Cozy and large room in the beautiful district ...,Berlin Berlin Germany,1,Prenzlauer Berg,1.0,1,...,10.0,9.0,1,0,0,moderate,0,0,1,1.75


In [336]:
#Se pueden codificar algunas columnas por diferentes niveles segun la respuesta de los clientes
#Politicas de cancelacion admite tres niveles: flexible, moderado y estricto 
def encode_politica_cancelacion(element: str) -> int:
    
    if element == 'flexible':
        return 0
    elif element == 'moderate':
        return 1
    elif element == 'strict_14_with_grace_period':
        return 2
    else:
        return 3

data_61['Reglas_cancelacion'] = data_61['Politica_Cancelacion'].apply(encode_politica_cancelacion)

In [337]:
data_61.head()

,ID,Nombre_Localidad,Opinion_Cliente,Espacio,Descripcion_Vivienda,Ubicacion_Anfitrion,Anfitrion_acojedor,Zona_Vivienda,Conteo_Total_Host,Anfitrion_Foto_Perfil,...,Requiere_Licencia,Reserva_Instantanea,Disponible_Viaje_Negocios,Politica_Cancelacion,Cliente_Foto_Perfil,Cliente_Telefono_vericicacion,Conteo_Lista_Anfitrion_Calculada,Reseña_Mensual,politica_cancelacion,Reglas_cancelacion
0,2015,Berlin Mitte Value Quiet courtyard very central,Great location 30 of 75 sq meters This wood...,A location This Einliegerwohnung is an e...,Great location 30 of 75 sq meters This wood...,Key Biscayne Florida United States,1,Mitte,4.0,1,...,1,0,0,strict_14_with_grace_period,0,0,4,3.76,2,2
1,2695,Prenzlauer Berg close to Mauerpark,Not specified,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,Berlin Berlin Germany,0,Prenzlauer Berg,1.0,1,...,1,0,0,flexible,0,0,1,1.42,0,0
2,3176,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor 68m2 apartment on Kollwitzplatz P...,This beautiful first floor apartment is situa...,Coledale New South Wales Australia,0,Prenzlauer Berg,1.0,1,...,1,1,0,strict_14_with_grace_period,0,0,1,1.25,2,2
3,3309,BerlinSpot Schöneberg near KaDeWe,First of all I prefer short notice bookings ...,Your room is really big and has 26 sqm is ver...,First of all I prefer short notice bookings ...,Berlin Berlin Germany,0,Schöneberg,1.0,1,...,1,0,0,strict_14_with_grace_period,0,0,1,0.39,2,2
4,7071,BrightRoom with sunny greenview,Cozy and large room in the beautiful district ...,The BrightRoom is an approx 20 sqm 215ft² ...,Cozy and large room in the beautiful district ...,Berlin Berlin Germany,1,Prenzlauer Berg,1.0,1,...,1,0,0,moderate,0,0,1,1.75,1,1


In [351]:
#Eliminar columnas politica de cancelacion porque habiamos creado la de reglas de cancelacion
def eliminar_columnas_politicas(df):
    
    New_DF_Airbnb = data_61.copy()
    
    New_DF_Airbnb.drop(['Politica_Cancelacion', 'politica_cancelacion'], axis = 1, inplace = True)
    
    return New_DF_Airbnb

New_DF_Airbnb = eliminar_columnas_politicas(data_61)

In [352]:
New_DF_Airbnb.head()

,ID,Nombre_Localidad,Opinion_Cliente,Espacio,Descripcion_Vivienda,Ubicacion_Anfitrion,Anfitrion_acojedor,Zona_Vivienda,Conteo_Total_Host,Anfitrion_Foto_Perfil,...,Ubicacion_Puntaje,Valor_Puntaje,Requiere_Licencia,Reserva_Instantanea,Disponible_Viaje_Negocios,Cliente_Foto_Perfil,Cliente_Telefono_vericicacion,Conteo_Lista_Anfitrion_Calculada,Reseña_Mensual,Reglas_cancelacion
0,2015,Berlin Mitte Value Quiet courtyard very central,Great location 30 of 75 sq meters This wood...,A location This Einliegerwohnung is an e...,Great location 30 of 75 sq meters This wood...,Key Biscayne Florida United States,1,Mitte,4.0,1,...,10.0,9.0,1,0,0,0,0,4,3.76,2
1,2695,Prenzlauer Berg close to Mauerpark,Not specified,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,Berlin Berlin Germany,0,Prenzlauer Berg,1.0,1,...,10.0,10.0,1,0,0,0,0,1,1.42,0
2,3176,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor 68m2 apartment on Kollwitzplatz P...,This beautiful first floor apartment is situa...,Coledale New South Wales Australia,0,Prenzlauer Berg,1.0,1,...,10.0,9.0,1,1,0,0,0,1,1.25,2
3,3309,BerlinSpot Schöneberg near KaDeWe,First of all I prefer short notice bookings ...,Your room is really big and has 26 sqm is ver...,First of all I prefer short notice bookings ...,Berlin Berlin Germany,0,Schöneberg,1.0,1,...,9.0,9.0,1,0,0,0,0,1,0.39,2
4,7071,BrightRoom with sunny greenview,Cozy and large room in the beautiful district ...,The BrightRoom is an approx 20 sqm 215ft² ...,Cozy and large room in the beautiful district ...,Berlin Berlin Germany,1,Prenzlauer Berg,1.0,1,...,10.0,9.0,1,0,0,0,0,1,1.75,1


In [353]:
#Guardamos el dataframe en formato csv
New_DF_Airbnb.to_csv("Airbnb_DataFrame.csv")